In [7]:
import pandas as pd
import numpy as np

import json
import sys
from pathlib import Path
from neurokit2 import signal_resample
from collections import Counter

def make_dict_array(atr_sample, atr_symbol):
    array_of_dict = []
    for sample, symbol in zip(atr_sample, atr_symbol):
        d = {'sample': sample, 'symbol': symbol}
        array_of_dict.append(d)
    return array_of_dict   

def print_dict_array(dict_array):
    for d in dict_array:
        sample = d['sample']
        symbol = d['symbol']
        if (symbol != 'N'):
            print(f"Sample: {sample}, Symbol: {symbol}")     

def zive_read_file_1ch(filename):
    f = open(filename, "r")
    a = np.fromfile(f, dtype=np.dtype('>i4'))
    ADCmax=0x800000
    Vref=2.5
    b = (a - ADCmax/2)*2*Vref/ADCmax/3.5*1000
    ecg_signal = b - np.mean(b)
    return ecg_signal

def zive_read_df_rpeaks(db_path, file_name):
    file_path = Path(db_path, file_name + '.json')
    with open(file_path,'r', encoding="utf8") as f:
        data = json.loads(f.read())
    df_rpeaks = pd.json_normalize(data, record_path =['rpeaks'])
    return df_rpeaks

def read_signl(rec_dir, filename):
    """
    Tinka EKG įrašų skaitymui tiek zive, tiek mit2zive atveju.
    zive atveju filename pvz. 1621694.321, 1621694.321.json
    mit2zive atveju, pvz. 100.000, 100.000.json - dalis iki taško ne ilgesnė
    už 4 simbolius

    Parameters
    ------------
        rec_dir: string
        filename: string
    Return
    -----------
        signl: numpy array, float
    """   
    file_path = Path(filename)
    name = file_path.stem
    extension = file_path.suffix
    print("Name:", name)
    print("Extension:", extension)

    file_path = Path(rec_dir, filename)
    print('file_path:', file_path)
    
    if len(name) <= 4:
        with open(file_path, "rb") as f:
            signl_loaded = np.load(f) 
        print(signl_loaded[:10])
        print('mit2zive signl_loaded ', signl_loaded.shape)
        return signl_loaded
    else:        
        signl_loaded = zive_read_file_1ch(file_path)
        print(signl_loaded[:10])
        print('zive signl_loaded', signl_loaded.shape)
        return signl_loaded


print("\nSukuriamas Zive duomenų analogas iš MIT duomenų")

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI\\DUOM_2022_RUDUO'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/DUOMENU_TVARKYMAS_2023'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  Aplankas, kur yra MIT2ZIVE duomenis
db_folder_mit2zive = 'records_npy_tst'


# Užduodamas pacientų įrašų sąrašas
# records_nr = np.array([124,209,228])
records_nr = np.array([100, 101, 103, 105, 106, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124,
200, 201, 202, 203, 205, 208, 209, 210, 212, 213, 214, 215, 219, 220, 221, 222, 223, 228, 230, 231, 232, 233, 234])

# Testavimui
records_nr = np.array([100.000, 100.001, 100.002])
records_nr = np.array([100.001, 1625402.027])

print("Bendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Pacientų įrašų sąrašas:\n",records_nr)
print("Aplankas, kur yra MIT2ZIVE duomenys:\n", db_folder_mit2zive)

print("Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers ir yra praleisti")
print("207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF")

# //////////////// Nurodomi MIT2ZIVE parametrai ////////////////////////////////

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais ir anotacijoms (.json)
rec_dir = Path(Duomenu_aplankas, db_folder_mit2zive)

        


Sukuriamas Zive duomenų analogas iš MIT duomenų
OS in my system :  linux
Bendras Zive duomenų aplankas:  /home/kesju/DI/DUOMENU_TVARKYMAS_2023
Pacientų įrašų sąrašas:
 [1.00001000e+02 1.62540203e+06]
Aplankas, kur yra MIT2ZIVE duomenys:
 records_npy_tst
Pastaba: įrašai su nr. 102, 104, 107, ir 217 yra gauti iš pacientų su pacemakers ir yra praleisti
207 paciento įrašas turi segmentus su ventricular flutter or fibrillation VF


In [8]:
print("\nKontrolinis skaitymas") 

# Kontrolinis  skaitymas

for filename in records_nr:
    # rec_ext = '{:03d}'.format(sub_recId)
    number_float = float(filename)
    filename = '{:.3f}'.format(number_float)
    print(filename)

    # Įrašas
    # Universalus variantas
    signl_loaded = read_signl(rec_dir, filename)

    # json
    # Nuskaitome gydytojo koreguotus EKG įrašo atributus iš įrašo json 
    df_rpeaks = zive_read_df_rpeaks(rec_dir, filename)
    rpeaks_from_json = df_rpeaks['sampleIndex'].to_numpy()
    symbols_from_json = df_rpeaks['annotationValue'].to_numpy()
    print(rpeaks_from_json[:20], symbols_from_json[:20])

    print("\nMakro anotacijų pasiskirstymas. Variantas iš json failo:")
    smb_lst = Counter(symbols_from_json)
    print(dict(smb_lst))
    print()
    # dict_array = make_dict_array(atr_sample,atr_symbol)
    # print_dict_array(dict_array)


print("\nPabaiga.............")



Kontrolinis skaitymas
100.001
Name: 100
Extension: .001
file_path: /home/kesju/DI/DUOMENU_TVARKYMAS_2023/records_npy_tst/100.001
[-0.29133333 -0.31433334 -0.30166665 -0.31966668 -0.32466663 -0.305
 -0.31800005 -0.31833335 -0.31266665 -0.31633336]
mit2zive signl_loaded  (120370,)
[120394 120550 120704 120859 121013 121178 121342 121501 121651 121803
 121960 122118 122276 122439 122606 122765 122917 123070 123224 123378] ['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N']

Makro anotacijų pasiskirstymas. Variantas iš json failo:
Counter({'N': 744, 'S': 12})

1625402.027
Name: 1625402
Extension: .027
file_path: /home/kesju/DI/DUOMENU_TVARKYMAS_2023/records_npy_tst/1625402.027
[-2.01291624 -2.16771802 -2.29578286 -2.40204943 -2.48617712 -2.55123134
 -2.59567937 -2.62275691 -2.63927591 -2.64847206]
zive signl_loaded (127999,)
[ 188  346  507  667  828  985 1151 1323 1495 1679 1858 2041 2219 2392
 2557 2725 2895 3074 3253 3439] ['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'